# <center>Anime Data Cleanup</center>

This Notebook taxes the json results of a Scrapy webscraper scraping https://myanimelist.net, cleans and fortmats the data and then produces useable csv files for future analysis. It is important to note that this is only data from show of the TV category (no Movies or OVA's). The webscraper ran on January 20th, 2017. So any titles or changes in ratings since then wil not be a part of this data set.

In [1]:
import json
from pprint import pprint
import numpy as np
import pandas as pd
from IPython.display import display, HTML
import re
with open('anime.json') as json_data:
    raw_json = json.load(json_data)

## Table of Contents
1. [Loading Raw Data](#loading)<br>
2. [Removing Missing Data](#missing)<br>
3. [Cleaning Data Tables](#cleaning)<br>
4. [Final Data Tables](#final)<br>

<a id ='loading'></a>

## Loading Raw Data From anime.json
<a id ='loading'></a>

In [2]:
listdata_json =[]
pagedata_json =[]
recommendationdata_json =[]
statsdata_json = []

In [3]:
for json in raw_json:
    if(json['pagetype'] == 'ListEntry'):
        listdata_json.append(json)
    elif(json['pagetype'] == 'PageEntry'):
        pagedata_json.append(json)
    elif(json['pagetype'] == 'StatsEntry'):
        statsdata_json.append(json)
    elif(json['pagetype'] == 'RecommendationEntry'):
        recommendationdata_json.append(json)

In [4]:
listframe = pd.DataFrame(listdata_json)
pageframe = pd.DataFrame(pagedata_json)
statsframe = pd.DataFrame(statsdata_json)
recommendationframe = pd.DataFrame(recommendationdata_json)

In [5]:
#Displays columns for each table
print("List Columns")
display(listframe.head(0))
print("Page Columns")
display(pageframe.head(0))
print("Stats Columns")
display(statsframe.head(0))
print("Recommendation Columns")
display(recommendationframe.head(0))

List Columns


,episodes,link,members,pagetype,rank,score,title


Page Columns


,aired,episodes,genres,pagetype,producers,recommendationlink,score,statslink,status,streamtype,studios,title


Stats Columns


,pagetype,score1,score10,score2,score3,score4,score5,score6,score7,score8,score9,title


Recommendation Columns


,pagetype,recommendations,title


## Removing  Incomplete/Missing Entries From Frames/Tables <a id='missing'></a>

In [6]:
#Checks integerty of array of iterables based on a column
def data_is_complete(arr, column):
    if(len(arr)==0):
        return False
    else:
        d = arr[0]
        for x in arr:
            if(not d[column].isin(x[column]).all()):
                return False
    return True

Now we have a set of tables(dataframes) corresponding to the data we scraped from the different types of pages. Now it's time to check if there are any shows that have missing entries across the tables. The method above serves that purpose. 'title' is the unique identifier across all shows.

In [7]:
print("Data Is Complete Across All Tables by Title: ", data_is_complete([statsframe,listframe,pageframe,recommendationframe], "title"))

Data Is Complete Across All Tables by Title:  False


As we can see the data is not consistent. Time to check what data we are missing. The cell bellow displays the size of each table, and then the number of titles missing from the ListEntries which is our source list. None of the other pages would have been scraped without out it. Note that a difference in size does not nessesarily have to equal the number of missing titles. Since they are equal (by looking below) this means there are no duplicates and we can be confident that we can remove the missing entries from List Entries without worries about consistency<br>

In [8]:
#Printing out lengths, and number of entries missing from the initial list to guide clean up
print("# of ListEntries:", len(listframe))
print("# of PageEntries (missing):", len(pageframe),"("+str(len(listframe)- len(pageframe))+")")
print("# of StatsEntries (missing):", len(statsdata_json),"("+str(len(listdata_json)- len(statsframe))+")")
print("# of RecommendationEntries (missing):", "("+str(len(listframe)- len(recommendationframe))+")")

# of ListEntries: 3830
# of PageEntries (missing): 3818 (12)
# of StatsEntries (missing): 3805 (25)
# of RecommendationEntries (missing): (20)


In [9]:
missing_page = (listframe[~listframe.title.isin(pageframe.title)])
missing_stats = (listframe[~listframe.title.isin(statsframe.title)])
missing_recommendation = (listframe[~listframe.title.isin(recommendationframe.title)])

In [10]:
#The missing number of page,stat,recommendation data corresponds the the numbers above. That means we don't have to worry about 
print('# Missing from PageEntries:', len(missing_page))
print('# Missing from StatsEntries:', len(missing_stats))
print('# Missing from RecommendationEntries:',len(missing_recommendation))

# Missing from PageEntries: 12
# Missing from StatsEntries: 25
# Missing from RecommendationEntries: 20


In [11]:
display(missing_stats[~missing_stats.title.isin(missing_recommendation.title)])
display(missing_recommendation[~missing_recommendation.title.isin(missing_page.title)])

,episodes,link,members,pagetype,rank,score,title
940,TV (13 eps),https://myanimelist.net/anime/17080/Soukyuu_no...,"12,182 members",ListEntry,941,7.44,Soukyuu no Fafner: Dead Aggressor - Exodus
961,TV (12 eps),https://myanimelist.net/anime/31771/Amanchu,"43,207 members",ListEntry,962,7.42,Amanchu!
965,TV (24 eps),https://myanimelist.net/anime/10444/Digimon_Xr...,"12,827 members",ListEntry,966,7.42,Digimon Xros Wars: Aku no Death General to Nan...
967,TV (12 eps),https://myanimelist.net/anime/21681/Hanayamata,"57,131 members",ListEntry,968,7.42,Hanayamata
971,TV (26 eps),https://myanimelist.net/anime/335/Matantei_Lok...,"25,450 members",ListEntry,972,7.42,Matantei Loki Ragnarok
985,TV (14 eps),https://myanimelist.net/anime/147/Kimi_ga_Nozo...,"91,721 members",ListEntry,986,7.41,Kimi ga Nozomu Eien
1048,TV (26 eps),https://myanimelist.net/anime/411/Gun_x_Sword,"46,499 members",ListEntry,1049,7.37,Gun x Sword
2718,TV (12 eps),https://myanimelist.net/anime/1885/Master_of_E...,"1,153 member",ListEntry,2719,6.24,Master of Epic: The Animation Age
2724,TV (53 eps),https://myanimelist.net/anime/5287/Ippatsu_Kan...,318 members,ListEntry,2725,6.23,Ippatsu Kanta-kun
2780,TV (26 eps),https://myanimelist.net/anime/2575/Don_Chuck_M...,429 members,ListEntry,2781,6.13,Don Chuck Monogatari


,episodes,link,members,pagetype,rank,score,title
935,TV (12 eps),https://myanimelist.net/anime/14833/Maoyuu_Mao...,"169,777 members",ListEntry,936,7.44,Maoyuu Maou Yuusha
944,TV (13 eps),https://myanimelist.net/anime/20787/Black_Bullet,"332,814 members",ListEntry,945,7.43,Black Bullet
945,TV (26 eps),https://myanimelist.net/anime/653/Galaxy_Angel_3,"6,307 members",ListEntry,946,7.43,Galaxy Angel 3
977,TV (5 eps),https://myanimelist.net/anime/1292/Afro_Samurai,"138,989 members",ListEntry,978,7.41,Afro Samurai
981,TV (24 eps),https://myanimelist.net/anime/287/Grappler_Bak...,"23,755 members",ListEntry,982,7.41,Grappler Baki (TV)
2760,TV (12 eps),https://myanimelist.net/anime/624/Mouse,"11,161 members",ListEntry,2761,6.17,Mouse
2765,TV (12 eps),https://myanimelist.net/anime/32171/Ange_Vierge,"23,206 members",ListEntry,2766,6.16,Ange Vierge
2778,TV (13 eps),https://myanimelist.net/anime/30851/Q_Transfor...,222 members,ListEntry,2779,6.14,Q Transformers: Saranaru Ninki Mono e no Michi


In [12]:
titles_to_remove = set(missing_stats.title.append(missing_page.title).append(missing_recommendation.title))

In [13]:
print('Titles with Incomplete Data:\n')
print(titles_to_remove)

Titles with Incomplete Data:

{'Mouse', 'Idol Memories', 'Digimon Xros Wars: Aku no Death General to Nanatsu no Oukoku', 'Amanchu!', 'Black Bullet', 'Hanayamata', 'Galaxy Angel 3', 'Jinki:Extend', 'Soukyuu no Fafner: Dead Aggressor - Exodus', 'Naked Wolves', 'Don Chuck Monogatari', 'Tari Tari', 'Mahou Shoujo Nante Mou Ii Desukara. 2nd Season', 'Gun x Sword', 'Cheating Craft', 'Master of Epic: The Animation Age', 'Ippatsu Kanta-kun', 'Manga Sekai Mukashibanashi', 'Q Transformers: Saranaru Ninki Mono e no Michi', 'Nabari no Ou', 'Ai Yori Aoshi: Enishi', 'Matantei Loki Ragnarok', 'Neko nanka Yondemo Konai.', 'Pan de Peace!', 'Mahoutsukai Chappy', 'Maoyuu Maou Yuusha', 'Kimi ga Nozomu Eien', 'Ayashi no Ceres', 'Grappler Baki (TV)', 'Ange Vierge', 'Afro Samurai', 'Mori no Tonto-tachi', 'D.C.II S.S.: Da Capo II Second Season'}


In [14]:
#Removing the titles from our frames
listframe = listframe[~listframe.title.isin(titles_to_remove)]
pageframe = pageframe[~pageframe.title.isin(titles_to_remove)]
statsframe = statsframe[~statsframe.title.isin(titles_to_remove)]
recommendationframe = recommendationframe[~recommendationframe.title.isin(titles_to_remove)]

In [15]:
print("Data Is Complete Across All Tables by Title: ",data_is_complete([statsframe,listframe,pageframe,recommendationframe], "title"))

Data Is Complete Across All Tables by Title:  True


<a id ='cleaning'></a>

## Cleaning Data Tables

Now that we got rid of the missing anime titles, its time to clean the data across and hopefully reduce the total number of tables to make analysis easier.

In [16]:
#Displays columns for each table
display(listframe.head(0))
display(pageframe.head(0))
display(statsframe.head(0))
display(recommendationframe.head(0))

,episodes,link,members,pagetype,rank,score,title


,aired,episodes,genres,pagetype,producers,recommendationlink,score,statslink,status,streamtype,studios,title


,pagetype,score1,score10,score2,score3,score4,score5,score6,score7,score8,score9,title


,pagetype,recommendations,title


#### Cleaning List Frame

In [17]:
listframe.head()

,episodes,link,members,pagetype,rank,score,title
0,TV (64 eps),https://myanimelist.net/anime/5114/Fullmetal_A...,"824,466 members",ListEntry,1,9.26,Fullmetal Alchemist: Brotherhood
1,TV (51 eps),https://myanimelist.net/anime/28977/Gintama°,"121,675 members",ListEntry,2,9.25,Gintama°
2,TV (? eps),https://myanimelist.net/anime/34096/Gintama,"32,296 members",ListEntry,3,9.17,Gintama.
3,TV (24 eps),https://myanimelist.net/anime/9253/Steins_Gate,"702,136 members",ListEntry,4,9.17,Steins;Gate
4,TV (51 eps),https://myanimelist.net/anime/9969/Gintama,"156,924 members",ListEntry,5,9.16,Gintama'


In [18]:
listframe =listframe.rename(index = str, columns={'rank':'ranking'}) #rank is an inherit method of dataframes so we're renaming it
listframe.ranking = listframe.ranking.apply(lambda x : int(x) if x.isdigit() else None)
listframe.members = listframe.members.str.replace(' member','').str.replace('s','').str.replace(',','').astype(int)

In [19]:
listframe.head()

,episodes,link,members,pagetype,ranking,score,title
0,TV (64 eps),https://myanimelist.net/anime/5114/Fullmetal_A...,824466,ListEntry,1.0,9.26,Fullmetal Alchemist: Brotherhood
1,TV (51 eps),https://myanimelist.net/anime/28977/Gintama°,121675,ListEntry,2.0,9.25,Gintama°
2,TV (? eps),https://myanimelist.net/anime/34096/Gintama,32296,ListEntry,3.0,9.17,Gintama.
3,TV (24 eps),https://myanimelist.net/anime/9253/Steins_Gate,702136,ListEntry,4.0,9.17,Steins;Gate
4,TV (51 eps),https://myanimelist.net/anime/9969/Gintama,156924,ListEntry,5.0,9.16,Gintama'


In [20]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
new_episodes = listframe.episodes.str.replace('TV','').str.replace('(','').str.replace(')','').str.replace('eps','')
new_episodes = new_episodes.apply(lambda x : int(x) if x.strip().isdigit() else None)
listframe.episodes = new_episodes
listframe.score = listframe.score.apply(lambda x : float(x) if is_number(x.strip()) else None)

In [21]:
listframe = listframe[['title','ranking','score','members','episodes','link']] #Rearrange columns and remove pagetype
display(listframe.head())

,title,ranking,score,members,episodes,link
0,Fullmetal Alchemist: Brotherhood,1.0,9.26,824466,64.0,https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama°,2.0,9.25,121675,51.0,https://myanimelist.net/anime/28977/Gintama°
2,Gintama.,3.0,9.17,32296,NaN,https://myanimelist.net/anime/34096/Gintama
3,Steins;Gate,4.0,9.17,702136,24.0,https://myanimelist.net/anime/9253/Steins_Gate
4,Gintama',5.0,9.16,156924,51.0,https://myanimelist.net/anime/9969/Gintama


#### Cleaning Recommendation Frame

In [22]:
display(recommendationframe.head())

,pagetype,recommendations,title
0,RecommendationEntry,"{'Noragami': '0', 'SKET Dance': '0', 'Sakigake...",Gintama': Enchousen
1,RecommendationEntry,"{'MÄR': '0', 'Chihayafuru': '0', 'Naruto': '17...",Hunter x Hunter (2011)
2,RecommendationEntry,"{'Monogatari Series: Second Season': '0', 'Roz...",Steins;Gate
3,RecommendationEntry,{},Gintama.
4,RecommendationEntry,"{'Baka to Test to Shoukanjuu': '1', 'Trigun': ...",Gintama°


Change the values in recommendations dictionaries from a string to an integer of <b>recommendationsframe</b>

In [23]:
for index, row in recommendationframe.iterrows():
    for key, value in row['recommendations'].items():
        row['recommendations'][key] = int(value)

In [24]:
recommendationframe = recommendationframe[['title','recommendations']] #Rearrange columns and remove pagetype

In [25]:
display(recommendationframe.head())

,title,recommendations
0,Gintama': Enchousen,"{'Noragami': 0, 'SKET Dance': 0, 'Sakigake!! C..."
1,Hunter x Hunter (2011),"{'MÄR': 0, 'Chihayafuru': 0, 'Naruto': 17, 'Dr..."
2,Steins;Gate,"{'Monogatari Series: Second Season': 0, 'Rozen..."
3,Gintama.,{}
4,Gintama°,"{'Baka to Test to Shoukanjuu': 1, 'Trigun': 0,..."


#### Cleaning Page Frame
Here is how the elements of our frame look initially, we need to remove the unessecarry html tags, and turn studios, producers and genres into a list

In [26]:
display(pageframe.head())

,aired,episodes,genres,pagetype,producers,recommendationlink,score,statslink,status,streamtype,studios,title
0,"Oct 4, 2012 to Mar 28, 2013",13,"<a href=""/anime/genre/1/Action"" title=""Action""...",PageEntry,"<a href=""/anime/producer/16/TV_Tokyo"" title=""T...",https://myanimelist.net/anime/15417/Gintama__E...,"<span itemprop=""ratingValue"">9.10</span><sup>1...",https://myanimelist.net/anime/15417/Gintama__E...,Finished Airing,"<a href=""https://myanimelist.net/topanime.php?...","<a href=""/anime/producer/14/Sunrise"" title=""Su...",Gintama': Enchousen
1,"Apr 4, 2011 to Mar 26, 2012",51,"<a href=""/anime/genre/1/Action"" title=""Action""...",PageEntry,"<a href=""/anime/producer/16/TV_Tokyo"" title=""T...",https://myanimelist.net/anime/9969/Gintama/use...,"<span itemprop=""ratingValue"">9.16</span><sup>1...",https://myanimelist.net/anime/9969/Gintama/stats,Finished Airing,"<a href=""https://myanimelist.net/topanime.php?...","<a href=""/anime/producer/14/Sunrise"" title=""Su...",Gintama'
2,"Oct 2, 2011 to Sep 24, 2014",148,"<a href=""/anime/genre/1/Action"" title=""Action""...",PageEntry,"<a href=""/anime/producer/29/VAP"" title=""VAP"">V...",https://myanimelist.net/anime/11061/Hunter_x_H...,"<span itemprop=""ratingValue"">9.13</span><sup>1...",https://myanimelist.net/anime/11061/Hunter_x_H...,Finished Airing,"<a href=""https://myanimelist.net/topanime.php?...","<a href=""/anime/producer/11/Madhouse"" title=""M...",Hunter x Hunter (2011)
3,"Apr 6, 2011 to Sep 14, 2011",24,"<a href=""/anime/genre/24/Sci-Fi"" title=""Sci-Fi...",PageEntry,"<a href=""/anime/producer/61/Frontier_Works"" ti...",https://myanimelist.net/anime/9253/Steins_Gate...,"<span itemprop=""ratingValue"">9.17</span><sup>1...",https://myanimelist.net/anime/9253/Steins_Gate...,Finished Airing,"<a href=""https://myanimelist.net/topanime.php?...","<a href=""/anime/producer/314/White_Fox"" title=...",Steins;Gate
4,"Jan 9, 2017 to ?",Unknown,"<a href=""/anime/genre/1/Action"" title=""Action""...",PageEntry,"<a href=""/anime/producer/14/Sunrise"" title=""Su...",https://myanimelist.net/anime/34096/Gintama/us...,"<span itemprop=""ratingValue"">9.17</span><sup>1...",https://myanimelist.net/anime/34096/Gintama/stats,Currently Airing,"<a href=""https://myanimelist.net/topanime.php?...","<a href=""/anime/producer/1258/Bandai_Namco_Pic...",Gintama.


Remove html tags from studios and genres and producers of <b>pageframe</b>

In [27]:
def extract_studios(s):
    if('None found' in s):
        return []
    return ([(lambda x: None if (('None found') in x or len(re.findall('<a.*>(.*)</a>', x))<1) else re.findall('<a.*>(.*)</a>', x)[0])(x) for x in s.split(',')])
pageframe.studios = pageframe.studios.apply(extract_studios)

def extract_genres(s):
    if len(re.findall('<a.*>(.*)</a>', s))<1:
        return []
    return ([(lambda x: None if len(re.findall('<a.*>(.*)</a>', x))<1 else re.findall('<a.*>(.*)</a>', x)[0])(x) for x in s.split(',')])
pageframe.genres = pageframe.genres.apply(extract_genres)

def extract_producers(s):
    if('None found' in s):
        return []
    return ([(lambda x: None if len(re.findall('<a.*>(.*)</a>', x))<1 else re.findall('<a.*>(.*)</a>', x)[0])(x) for x in s.split(',')])
pageframe.producers = pageframe.producers.apply(extract_producers)


In [28]:
pageframe = pageframe.drop(['recommendationlink', 'statslink', 'streamtype'], axis =1) #Drop columns with useless info
pageframe.head() #show partially cleaned page frame

,aired,episodes,genres,pagetype,producers,score,status,studios,title
0,"Oct 4, 2012 to Mar 28, 2013",13,"[Action, Comedy, Historical, Parody, Samurai, ...",PageEntry,"[TV Tokyo, Aniplex, Dentsu, Shueisha, Miracle ...","<span itemprop=""ratingValue"">9.10</span><sup>1...",Finished Airing,[Sunrise],Gintama': Enchousen
1,"Apr 4, 2011 to Mar 26, 2012",51,"[Action, Sci-Fi, Comedy, Historical, Parody, S...",PageEntry,"[TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mir...","<span itemprop=""ratingValue"">9.16</span><sup>1...",Finished Airing,[Sunrise],Gintama'
2,"Oct 2, 2011 to Sep 24, 2014",148,"[Action, Adventure, Super Power, Shounen]",PageEntry,"[VAP, Nippon Television Network Corporation, S...","<span itemprop=""ratingValue"">9.13</span><sup>1...",Finished Airing,[Madhouse],Hunter x Hunter (2011)
3,"Apr 6, 2011 to Sep 14, 2011",24,"[Sci-Fi, Thriller]",PageEntry,"[Frontier Works, Media Factory, Movic, AT-X, K...","<span itemprop=""ratingValue"">9.17</span><sup>1...",Finished Airing,[White Fox],Steins;Gate
4,"Jan 9, 2017 to ?",Unknown,"[Action, Sci-Fi, Comedy, Historical, Parody, S...",PageEntry,"[Sunrise, TV Tokyo]","<span itemprop=""ratingValue"">9.17</span><sup>1...",Currently Airing,[Bandai Namco Pictures],Gintama.


Remove html tag and split scores into two different columns for <b>pageframe</b> and also convert episodes into numeric column

In [29]:
pageframe.score = pageframe.score.apply(lambda s: (re.findall('<span.*>(.*)</span>.*<span.*>(.*)</span>.*',s))[0])
pageframe.score,pageframe['num_votes']=pageframe.score.apply(pd.Series)[0],pageframe.score.apply(pd.Series)[1]
pageframe.num_votes = pageframe.num_votes.str.replace(',','').astype(int)
pageframe.score = pageframe.score.apply(lambda x: None if x=='N/A' else x).astype(float)
pageframe.episodes = pageframe.episodes.apply(lambda x: int(x) if x.isdigit() else None)

In [30]:
display(pageframe.head())

,aired,episodes,genres,pagetype,producers,score,status,studios,title,num_votes
0,"Oct 4, 2012 to Mar 28, 2013",13.0,"[Action, Comedy, Historical, Parody, Samurai, ...",PageEntry,"[TV Tokyo, Aniplex, Dentsu, Shueisha, Miracle ...",9.10,Finished Airing,[Sunrise],Gintama': Enchousen,44356
1,"Apr 4, 2011 to Mar 26, 2012",51.0,"[Action, Sci-Fi, Comedy, Historical, Parody, S...",PageEntry,"[TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mir...",9.16,Finished Airing,[Sunrise],Gintama',67153
2,"Oct 2, 2011 to Sep 24, 2014",148.0,"[Action, Adventure, Super Power, Shounen]",PageEntry,"[VAP, Nippon Television Network Corporation, S...",9.13,Finished Airing,[Madhouse],Hunter x Hunter (2011),251094
3,"Apr 6, 2011 to Sep 14, 2011",24.0,"[Sci-Fi, Thriller]",PageEntry,"[Frontier Works, Media Factory, Movic, AT-X, K...",9.17,Finished Airing,[White Fox],Steins;Gate,403535
4,"Jan 9, 2017 to ?",NaN,"[Action, Sci-Fi, Comedy, Historical, Parody, S...",PageEntry,"[Sunrise, TV Tokyo]",9.17,Currently Airing,[Bandai Namco Pictures],Gintama.,6983


Parsing aired as two date columns, air_start and air_end

In [31]:
airseries = pageframe.aired
airseries.loc[(airseries == 'Not available')]=np.nan
airseries_split = airseries.str.split('to')

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
dateframe = airseries_split.apply(lambda x: [x[0],np.nan] if (isinstance(x,list) and len(x)<2) else (lambda y: [y,np.nan] if isinstance(y, float) else y)(x))
dateframe = dateframe.apply(pd.Series)
display(dateframe.head())

,0,1
0,"Oct 4, 2012","Mar 28, 2013"
1,"Apr 4, 2011","Mar 26, 2012"
2,"Oct 2, 2011","Sep 24, 2014"
3,"Apr 6, 2011","Sep 14, 2011"
4,"Jan 9, 2017",?


In [33]:
dateframe[0] = dateframe[0].str.strip().replace('?',np.nan)
dateframe[1] = dateframe[1].str.strip().replace('?',np.nan)
dateframe[0] = dateframe[0].apply(lambda x: pd.to_datetime(x))
dateframe[1] = dateframe[1].apply(lambda x: pd.to_datetime(x))
pageframe['air_start'],pageframe['air_end'] = dateframe[0], dateframe[1]

In [34]:
#Rearrange columns & remove page frame and aire
pageframe = pageframe[['title','episodes','score','num_votes','air_start','air_end','genres','producers','studios','status']]

In [35]:
display(pageframe.head())

,title,episodes,score,num_votes,air_start,air_end,genres,producers,studios,status
0,Gintama': Enchousen,13.0,9.10,44356,2012-10-04,2013-03-28,"[Action, Comedy, Historical, Parody, Samurai, ...","[TV Tokyo, Aniplex, Dentsu, Shueisha, Miracle ...",[Sunrise],Finished Airing
1,Gintama',51.0,9.16,67153,2011-04-04,2012-03-26,"[Action, Sci-Fi, Comedy, Historical, Parody, S...","[TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mir...",[Sunrise],Finished Airing
2,Hunter x Hunter (2011),148.0,9.13,251094,2011-10-02,2014-09-24,"[Action, Adventure, Super Power, Shounen]","[VAP, Nippon Television Network Corporation, S...",[Madhouse],Finished Airing
3,Steins;Gate,24.0,9.17,403535,2011-04-06,2011-09-14,"[Sci-Fi, Thriller]","[Frontier Works, Media Factory, Movic, AT-X, K...",[White Fox],Finished Airing
4,Gintama.,NaN,9.17,6983,2017-01-09,NaT,"[Action, Sci-Fi, Comedy, Historical, Parody, S...","[Sunrise, TV Tokyo]",[Bandai Namco Pictures],Currently Airing


### Cleaning Stats Frame

In [36]:
display(statsframe.head())
statsframe.info()

,pagetype,score1,score10,score2,score3,score4,score5,score6,score7,score8,score9,title
0,StatsEntry,396,23134,44,55,121,340,739,2619,6405,11860,Gintama': Enchousen
1,StatsEntry,147,29137,18,28,78,256,794,3380,11566,23703,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...
2,StatsEntry,340,47495,211,396,1034,3277,8412,30435,64263,64379,Kuroko no Basket
3,StatsEntry,44,10629,46,79,248,960,2750,9401,16087,13126,Kamisama Hajimemashita◎
4,StatsEntry,78,6341,55,120,354,1186,2699,7023,10723,8780,Hikaru no Go


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3804
Data columns (total 12 columns):
pagetype    3797 non-null object
score1      3797 non-null object
score10     3797 non-null object
score2      3797 non-null object
score3      3797 non-null object
score4      3797 non-null object
score5      3797 non-null object
score6      3797 non-null object
score7      3797 non-null object
score8      3797 non-null object
score9      3797 non-null object
title       3797 non-null object
dtypes: object(12)
memory usage: 385.6+ KB


In [37]:
#convert score to integer types
statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']]=statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']].astype(int)
statsframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3804
Data columns (total 12 columns):
pagetype    3797 non-null object
score1      3797 non-null int32
score10     3797 non-null int32
score2      3797 non-null int32
score3      3797 non-null int32
score4      3797 non-null int32
score5      3797 non-null int32
score6      3797 non-null int32
score7      3797 non-null int32
score8      3797 non-null int32
score9      3797 non-null int32
title       3797 non-null object
dtypes: int32(10), object(2)
memory usage: 237.3+ KB


In [38]:
#rearrange columns and remove pagetype
statsframe= statsframe[['title','score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']]
display(statsframe.head())

,title,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10
0,Gintama': Enchousen,396,44,55,121,340,739,2619,6405,11860,23134
1,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,147,18,28,78,256,794,3380,11566,23703,29137
2,Kuroko no Basket,340,211,396,1034,3277,8412,30435,64263,64379,47495
3,Kamisama Hajimemashita◎,44,46,79,248,960,2750,9401,16087,13126,10629
4,Hikaru no Go,78,55,120,354,1186,2699,7023,10723,8780,6341


In [39]:
#Create Average score
weight= np.array([1,2,3,4,5,6,7,8,9,10])
statsframe['num_votes'] =statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']].sum(axis=1)
weighted_sum = statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']].dot(weight)
statsframe['score'] = (weighted_sum/(statsframe.num_votes))
a = (statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']]**.5)*weight
b = (statsframe[['score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']]**.5).multiply(statsframe.score, axis='rows')
stdev = ((((a-b)**2).sum(axis=1))/statsframe.num_votes)**.5
statsframe['stdev'] = stdev

In [40]:
#rearrange columns and remove pagetype
statsframe = statsframe[['title','score','stdev','num_votes','score1','score2','score3','score4','score5','score6','score7','score8','score9','score10']]

In [41]:
statsframe.head()

,title,score,stdev,num_votes,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10
0,Gintama': Enchousen,9.076630,1.358325,45713,396,44,55,121,340,739,2619,6405,11860,23134
1,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,9.080238,1.064220,69107,147,18,28,78,256,794,3380,11566,23703,29137
2,Kuroko no Basket,8.420070,1.278095,220242,340,211,396,1034,3277,8412,30435,64263,64379,47495
3,Kamisama Hajimemashita◎,8.274162,1.301665,53370,44,46,79,248,960,2750,9401,16087,13126,10629
4,Hikaru no Go,8.069354,1.434932,37359,78,55,120,354,1186,2699,7023,10723,8780,6341


After looking through some of the data there seems to be some discrepancy between the calculated score in the <b>statsframe</b> and scores on <b>pageframe</b> which are the ones that are officially displayed on the webssite.

This is because of how My Anime List calculates its score.<i>
> The score of a show as indicated by MAL users. MAL uses a weighted score generated from users, the only thing people need to keep in mind is how the final score is reached. MAL only records the scores of users who have watched 1/5th of a series to calculate its final score, however in order to generate a score before it reaches this point it reads the score of all users and follows the '1/5th rule' only after the show is completed, as such you may notice disparities between the penultimate and final score of a show. <br> https://myanimelist.net/clubs.php?cid=59197</i>


Since we don't have user specific data we'll just have be aware while doing an analysis on the discrepency between the two.

<a id ='final'></a>
## Final Data Tables

In [42]:
display(HTML('<h4>List Frame</h4'))
display(listframe.head())
listframe.info()
display(HTML('<h4>Page Frame</h4'))
display(pageframe.head())
pageframe.info()
display(HTML('<h4>Recommendation Frame</h4'))
display(recommendationframe.head())
recommendationframe.info()
display(HTML('<h4>Stats Frame</h4'))
display(statsframe.head())
statsframe.info()

,title,ranking,score,members,episodes,link
0,Fullmetal Alchemist: Brotherhood,1.0,9.26,824466,64.0,https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama°,2.0,9.25,121675,51.0,https://myanimelist.net/anime/28977/Gintama°
2,Gintama.,3.0,9.17,32296,NaN,https://myanimelist.net/anime/34096/Gintama
3,Steins;Gate,4.0,9.17,702136,24.0,https://myanimelist.net/anime/9253/Steins_Gate
4,Gintama',5.0,9.16,156924,51.0,https://myanimelist.net/anime/9969/Gintama


<class 'pandas.core.frame.DataFrame'>
Index: 3797 entries, 0 to 3829
Data columns (total 6 columns):
title       3797 non-null object
ranking     3697 non-null float64
score       3694 non-null float64
members     3797 non-null int32
episodes    3583 non-null float64
link        3797 non-null object
dtypes: float64(3), int32(1), object(2)
memory usage: 192.8+ KB


,title,episodes,score,num_votes,air_start,air_end,genres,producers,studios,status
0,Gintama': Enchousen,13.0,9.10,44356,2012-10-04,2013-03-28,"[Action, Comedy, Historical, Parody, Samurai, ...","[TV Tokyo, Aniplex, Dentsu, Shueisha, Miracle ...",[Sunrise],Finished Airing
1,Gintama',51.0,9.16,67153,2011-04-04,2012-03-26,"[Action, Sci-Fi, Comedy, Historical, Parody, S...","[TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mir...",[Sunrise],Finished Airing
2,Hunter x Hunter (2011),148.0,9.13,251094,2011-10-02,2014-09-24,"[Action, Adventure, Super Power, Shounen]","[VAP, Nippon Television Network Corporation, S...",[Madhouse],Finished Airing
3,Steins;Gate,24.0,9.17,403535,2011-04-06,2011-09-14,"[Sci-Fi, Thriller]","[Frontier Works, Media Factory, Movic, AT-X, K...",[White Fox],Finished Airing
4,Gintama.,NaN,9.17,6983,2017-01-09,NaT,"[Action, Sci-Fi, Comedy, Historical, Parody, S...","[Sunrise, TV Tokyo]",[Bandai Namco Pictures],Currently Airing


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3817
Data columns (total 10 columns):
title        3797 non-null object
episodes     3583 non-null float64
score        3694 non-null float64
num_votes    3797 non-null int32
air_start    3760 non-null datetime64[ns]
air_end      3506 non-null datetime64[ns]
genres       3797 non-null object
producers    3797 non-null object
studios      3797 non-null object
status       3797 non-null object
dtypes: datetime64[ns](2), float64(2), int32(1), object(5)
memory usage: 311.5+ KB


,title,recommendations
0,Gintama': Enchousen,"{'Noragami': 0, 'SKET Dance': 0, 'Sakigake!! C..."
1,Hunter x Hunter (2011),"{'MÄR': 0, 'Chihayafuru': 0, 'Naruto': 17, 'Dr..."
2,Steins;Gate,"{'Monogatari Series: Second Season': 0, 'Rozen..."
3,Gintama.,{}
4,Gintama°,"{'Baka to Test to Shoukanjuu': 1, 'Trigun': 0,..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3809
Data columns (total 2 columns):
title              3797 non-null object
recommendations    3797 non-null object
dtypes: object(2)
memory usage: 89.0+ KB


,title,score,stdev,num_votes,score1,score2,score3,score4,score5,score6,score7,score8,score9,score10
0,Gintama': Enchousen,9.076630,1.358325,45713,396,44,55,121,340,739,2619,6405,11860,23134
1,Haikyuu!!: Karasuno Koukou VS Shiratorizawa Ga...,9.080238,1.064220,69107,147,18,28,78,256,794,3380,11566,23703,29137
2,Kuroko no Basket,8.420070,1.278095,220242,340,211,396,1034,3277,8412,30435,64263,64379,47495
3,Kamisama Hajimemashita◎,8.274162,1.301665,53370,44,46,79,248,960,2750,9401,16087,13126,10629
4,Hikaru no Go,8.069354,1.434932,37359,78,55,120,354,1186,2699,7023,10723,8780,6341


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3797 entries, 0 to 3804
Data columns (total 14 columns):
title        3797 non-null object
score        3764 non-null float64
stdev        3764 non-null float64
num_votes    3797 non-null int64
score1       3797 non-null int32
score2       3797 non-null int32
score3       3797 non-null int32
score4       3797 non-null int32
score5       3797 non-null int32
score6       3797 non-null int32
score7       3797 non-null int32
score8       3797 non-null int32
score9       3797 non-null int32
score10      3797 non-null int32
dtypes: float64(2), int32(10), int64(1), object(1)
memory usage: 296.6+ KB


One last check on integrity of data just to make sure we didn't mess anything up

In [43]:
data_is_complete([statsframe,listframe,pageframe,recommendationframe],'title')

True

### Publishing Clean Data

In [44]:
listframe.to_csv('animelist.csv', encoding='utf-8',index= False)
pageframe.to_csv('animepage.csv', encoding='utf-8',index= False)
recommendationframe.to_csv('animerecommendation.csv', encoding='utf-8',index= False)
statsframe.to_csv('animestats.csv', encoding='utf-8',index= False)